## Imports

In [1]:
import os  # Biblioteca padrão do Python usada para interagir com o sistema operacional, como manipular arquivos e diretórios.
import openai  # Biblioteca para acessar a API da OpenAI, permitindo o uso de modelos de linguagem como GPT.
import langchain  # Framework para desenvolvimento de aplicativos baseados em linguagem natural, integra várias ferramentas de IA.
import chromadb  # Biblioteca para interagir com o ChromaDB, um sistema de armazenamento de embeddings vetoriais.
from langchain_community.document_loaders import PyPDFDirectoryLoader  # Carrega documentos PDF de um diretório para processamento.
from langchain_community.embeddings import OpenAIEmbeddings  # Facilita a criação e o uso de embeddings gerados pelos modelos da OpenAI.
from langchain_community.vectorstores import Chroma  # Interface para armazenar e recuperar embeddings no ChromaDB.
from langchain_community.llms import OpenAI  # Permite integrar e usar modelos de linguagem da OpenAI em cadeias de processamento de linguagem.
from langchain.chains.question_answering import load_qa_chain  # Carrega cadeias de processamento configuradas para question answering.
import warnings  # Biblioteca padrão do Python para gerenciar avisos.
warnings.filterwarnings('ignore')  # Configura a biblioteca de avisos para ignorar todos os avisos emitidos.

## Extraindo texto de documento PDF

In [2]:
def extracao_pdf(path):

    # instância da classe para leitura ds PDFs contidos na path
    leitor = PyPDFDirectoryLoader(path)

    # fazendo a leitura
    # Cada documento contém o texto e, possivelmente, metadados associados. Estes documentos são convertidos em vetores (embeddings) usando um modelo de 
    # embeddings e armazenados no índice
    documentos_lidos = leitor.load()

    return documentos_lidos

In [3]:
caminho = r'C:\Users\Vinicius Moraes\Google Drive\estudos\data science, machine learning, estatistica\DSA\ia generativa e llms\mod05\87-Cap05\arquivos2'
texto_artigo = extracao_pdf(caminho)

In [4]:
len(texto_artigo)

50

## Criando gerador de Embeddings

In [5]:
os.environ['OPENAI_API_KEY'] = 'API KEY'

In [6]:
gerador_embeddings = OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])

In [7]:
''' 
A função embed_query é utilizada para gerar embeddings de uma consulta de texto usando os modelos de embeddings da OpenAI. 
Esses embeddings podem ser usados para fins de busca, recuperação de informações, e outras aplicações de NLP 
'''
vetores = gerador_embeddings.embed_query("Me fale sobre o Flamengo")

In [8]:
vetores[:5]

[-0.015223479866010815,
 -0.008045822893365985,
 -0.0068536228199858225,
 -0.01039965423731675,
 -0.007672878469947157]

## Definindo o Vector Store

In [9]:
# criando o nome do indice
# index_nome = 'artigo'
index_nome = {"name": "artigo"}

In [10]:
texto_artigo

[Document(metadata={'source': 'C:\\Users\\Vinicius Moraes\\Google Drive\\estudos\\data science, machine learning, estatistica\\DSA\\ia generativa e llms\\mod05\\87-Cap05\\arquivos2\\2404.07544v2.pdf', 'page': 0}, page_content='Preprint. A version of this work is under review.\nFrom Words to Numbers: Your Large Language Model Is Se-\ncretly A Capable Regressor When Given In-Context Examples\nRobert Vacareanu∗\nUniversity of Arizona\nTechnical University of Cluj-NapocaVlad-Andrei Negru\nTechnical University of Cluj-Napoca\nVasile Suciu\nTechnical University of Cluj-NapocaMihai Surdeanu\nUniversity of Arizona\nAbstract\nWe analyze how well pre-trained large language models (e.g., Llama2,\nGPT-4, Claude 3, etc) can do linear and non-linear regression when given\nin-context examples, without any additional training or gradient updates.\nOur findings reveal that several large language models (e.g., GPT-4, Claude\n3) are able to perform regression tasks with a performance rivaling (or even\no

In [11]:
# função do LangChain que cria um índice vetorial (vector store) a partir de uma lista de documentos fornecidos. Esse índice vetorial permite buscas por 
# similaridade, onde documentos podem ser recuperados com base na similaridade de seus embeddings
# texto_artigo é uma lista de objetos Document ou textos que contêm o conteúdo dos documentos a serem armazenados no índice vetorial
index = Chroma.from_documents(documents=texto_artigo, embedding=gerador_embeddings, collection_metadata=index_nome)

In [12]:
index

## Definindo função para fazer a busca por similaridade

In [13]:
def busca_similaridade(query, k = 2):

    resultado_similar = index.similarity_search(query, k=k)

    return resultado_similar

## Criando o app do assistente pessoal com LangChain

In [14]:
llm = OpenAI(api_key=os.environ['OPENAI_API_KEY'], temperature = .3)

In [15]:
chain = load_qa_chain(llm, chain_type = 'stuff')

In [16]:
# Define a função para obter resposta
def obter_resposta(query):

    # Chama a função 'dsa_busca_similaridade' com a 'query' fornecida e armazena o resultado em 'doc_search'
    # Utiliza o método 'similarity_search' do objeto 'index' para buscar os 'k' resultados mais semelhantes à 'query'
    similar = busca_similaridade(query)

    # Utiliza o objeto 'chain' para executar a função run e processar a 'query' e os documentos encontrados, armazenando a resposta em 'response'
    # faz parte de um pipeline de perguntas e respostas c/ o framework LangChain, c/ objetivo de responder a uma pergunta (query) com base em documentos 
    # fornecidos
    resposta = chain.run(input_documents=similar, question=query)

    return resposta

## Executando o assistente de IA e conversando com PDFs

In [19]:
prompt1 = "sobre o que fala o documento? responda em português do Brasil"

In [20]:
resposta = obter_resposta(prompt1)
print(resposta)

 O documento fala sobre modelos de linguagem de grande escala (LLMs) e sua utilização em tarefas de processamento de linguagem natural (PLN). Ele apresenta uma análise dos diferentes LLMs disponíveis, suas características e desempenho em tarefas específicas. Além disso, o documento discute o impacto do pré-treinamento e do contexto no aprendizado desses modelos.
